<a href="https://colab.research.google.com/github/ahlqui/VeloxChemColabs/blob/main/BDEtohydrogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install conda (ca 1 min)
%%capture
!pip install -q condacolab
import condacolab
condacolab.install()


In [ ]:
#@title Install all the codes needed for this notebook (ca 1-2 min)
%%capture
!conda install -c conda-forge -c veloxchem veloxchem openbabel py3Dmol

import veloxchem as vlx

In [ ]:
xyz_coordinates = """
C          0.32160        0.72410       -0.00040
C         -0.90270        1.41980        0.00660
C         -2.10150        0.69710        0.01120
C         -2.09140       -0.69640        0.00890
C         -0.88830       -1.40910        0.00200
C          0.30120       -0.67090       -0.00250
C          1.68680        1.12310       -0.00630
C          2.45750       -0.01710       -0.01180
N          1.61120       -1.09430       -0.00950
H         -0.91840        2.50550        0.00840
H         -3.04870        1.23060        0.01660
H         -3.03380       -1.23930        0.01250
H         -0.88450       -2.49400        0.00020
H          2.06150        2.13810       -0.00640
H          3.53030       -0.15430       -0.01710
H          1.89930       -2.06280       -0.01240
"""

In [ ]:
#@title Define your molecule, either asa SMILES code or xyz-coordinates above
#@markdown - Enter the SMILES code
smiles_code = 'C1=CN=CN1' #@param {type:"string"}
#@markdown - If you want to use the xyz coordinates in the previous cell check box
use_xyz = False # @param {type:"boolean"}
if use_xyz == False:
  molecule = vlx.Molecule.read_smiles(smiles_code)
else:
  molecule = vlx.Molecule.read_str(xyz_coordinates)
print('Structure of the molecule entered: ')
molecule.show(atom_indices=True)

Structure of the molecule entered: 


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#@title Pre-optimization of the geometry with xTB
#@markdown Example timing: Ibuprofen (59 sec)
scf_drv = vlx.XtbDriver()
scf_drv.ostream.mute()

opt_drv = vlx.OptimizationDriver(scf_drv)
opt_drv.ostream.mute()
opt_results = opt_drv.compute(molecule)

molecule = vlx.Molecule.read_xyz_string(opt_results["final_geometry"])
print('xTB optimized geometry:')
molecule.show()


xTB optimized geometry:


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#@title Generate structure objects without one hydrogen atom each
def remove_atom_iteratively(molecule, atom):
    mol_string = molecule.get_xyz_string()
    number_of_atoms = molecule.number_of_atoms()
    mol_string = mol_string.split('\n')
    # Identify the lines that start with atom and save the positions
    atom_positions = []
    for i, line in enumerate(mol_string):
        if line.startswith(atom):
            atom_positions.append(i)
    # Remove the atom lines one by one and generate one molecule for each
    molecules = []
    for i, position in enumerate(atom_positions):
        new_mol = mol_string.copy()
        new_mol.pop(position)
        # Update the number of atoms
        new_mol[0] = str(number_of_atoms - 1)
        new_mol = '\n'.join(new_mol)
        molecules.append(vlx.Molecule.read_xyz_string(new_mol))
    return molecules
molecules = remove_atom_iteratively(molecule, 'H')
#for mol in molecules:
#    mol.show()


In [ ]:
#@title DFT single point
#@markdown Example timing: Indole (24 min) PBE/6-31G*
print('Computing energy of structure 1' + ' of ' + str(len(molecules)+1))
basis_set = '6-31G*' #@param  ['6-31++G', '6-31++G*', '6-31++G**', '6-31+G', '6-31+G*', '6-31+G**', '6-311++G', '6-311++G(2D,2P)', '6-311++G(3DF,3PD)', '6-311++G*', '6-311++G**', '6-311+G', '6-311+G(2D,P)', '6-311+G*', '6-311+G**', '6-311G', '6-311G(2DF,2PD)', '6-311G*', '6-311G**', '6-31G', '6-31G(2DF,P)', '6-31G(3DF,3PD)', '6-31G*', '6-31G**', 'ANO-L-MB', 'ANO-L-VDZP', 'ANO-L-VQZP', 'ANO-L-VTZP', 'ANO-S-MB', 'ANO-S-VDZP', 'AO-START-GUESS', 'AUG-CC-PCV5Z', 'AUG-CC-PCVDZ', 'AUG-CC-PCVQZ', 'AUG-CC-PCVTZ', 'AUG-CC-PV5Z', 'AUG-CC-PV6Z', 'AUG-CC-PVDZ', 'AUG-CC-PVQZ', 'AUG-CC-PVTZ', 'AUG-PCSEG-0', 'AUG-PCSEG-1', 'AUG-PCSEG-2', 'AUG-PCSEG-3', 'AUG-PCSEG-4', 'AUG-PCX-1', 'AUG-PCX-2', 'AUG-PCX-3', 'AUG-PCX-4', 'CC-PCV5Z', 'CC-PCVDZ', 'CC-PCVQZ', 'CC-PCVTZ', 'CC-PV5Z', 'CC-PV6Z', 'CC-PVDZ', 'CC-PVQZ', 'CC-PVTZ', 'DAUG-CC-PCV5Z', 'DAUG-CC-PCVDZ', 'DAUG-CC-PCVQZ', 'DAUG-CC-PCVTZ', 'DAUG-CC-PV5Z', 'DAUG-CC-PV6Z', 'DAUG-CC-PVDZ', 'DAUG-CC-PVQZ', 'DAUG-CC-PVTZ', 'DEF2-QZVP', 'DEF2-QZVPD', 'DEF2-QZVPP', 'DEF2-QZVPPD', 'DEF2-SV(P)', 'DEF2-SVP', 'DEF2-SVPD', 'DEF2-TZVP', 'DEF2-TZVPD', 'DEF2-TZVPP', 'DEF2-TZVPPD', 'PCSEG-0', 'PCSEG-1', 'PCSEG-2', 'PCSEG-3', 'PCSEG-4', 'PCX-1', 'PCX-2', 'PCX-3', 'PCX-4', 'SADLEJ-PVTZ', 'STO-3G', 'STO-3G-OLD', 'STO-6G', 'TAUG-CC-PCV5Z', 'TAUG-CC-PCVDZ', 'TAUG-CC-PCVQZ', 'TAUG-CC-PCVTZ', 'TAUG-CC-PV5Z', 'TAUG-CC-PV6Z', 'TAUG-CC-PVDZ', 'TAUG-CC-PVQZ', 'TAUG-CC-PVTZ']
basis = vlx.MolecularBasis.read(molecule, basis_set, ostream=None)
scf_drv = vlx.ScfUnrestrictedDriver()
mute_output = True # @param {type:"boolean"}
if mute_output == True:
  scf_drv.ostream.mute()
else:
  scf_drv.ostream
functional = 'PBE0' #@param ['SLATER', 'SLDA', 'B88X', 'BLYP', 'B3LYP', 'BHANDH', 'BHANDHLYP', 'PBE', 'PBE0', 'REVPBE', 'BP86', 'PW91', 'MPW1K', 'OLYP', 'O3LYP', 'X3LYP', 'B97', 'B97-1', 'B97-2', 'B97-3', 'TPSS', 'TPSSH', 'REVTPSS', 'PKZB', 'SCAN', 'RSCAN', 'R2SCAN', 'M05', 'M05-2X', 'M06', 'M06-2X', 'M06-HF', 'M06-L', 'M11-L', 'MPW1B95', 'MPWB1K', 'PW6B95', 'PWB6K']
scf_drv.xcfun = functional
scf_results = scf_drv.compute(molecule, basis)
print('Done')
#print(f"DFT energy: {scf_drv.get_scf_energy():14.10f} a.u.")

hydrogen = vlx.Molecule.read_str(""" H 0.0 0.0 0.0 """)
hydrogen.set_multiplicity(2)
basis = vlx.MolecularBasis.read(hydrogen, basis_set, ostream=None)
scf_resultsH = scf_drv.compute(hydrogen, basis)
#print(f"DFT energy: {scf_drv.get_scf_energy():14.10f} a.u.")
i = 1
results = []
for mol in molecules:
    i = i + 1
    print('Computing energy of structure ' + str(i) + ' of ' + str(len(molecules)+1))
    basis = vlx.MolecularBasis.read(mol, basis_set, ostream=None)
    mol.set_multiplicity(2)
    scf_resultsmol = scf_drv.compute(mol, basis)
    print('Done')
    results.append(scf_resultsmol)

Computing energy of structure 1 of 5
Done
Computing energy of structure 2 of 5
Done
Computing energy of structure 3 of 5
Done
Computing energy of structure 4 of 5
Done
Computing energy of structure 5 of 5
Done


In [ ]:
#@title Print BDEs and geometry
last_energies = []
bdes = []

for i in range(len(molecules)):
  res = results[i]['scf_energy']
  last_energies.append(res)
#for i in range(len(molecules)):
#  print('Energy', last_energies[i])

molecule.show(atom_indices=True)


for i in range(len(molecules)):
  bde =  scf_resultsH['scf_energy'] + last_energies[i] -scf_results['scf_energy']
  bdes.append(bde)
#for i in range(len(molecules)):
#  molecules[i].show()
#  print('BDE: ', 627.5*bdes[i])

print()
mol_string = molecule.get_xyz_string()
number_of_atoms = molecule.number_of_atoms()
mol_string = mol_string.split('\n')
# Identify the lines that start with atom and save the positions
atom_positions = []
for i, line in enumerate(mol_string):
    if line.startswith('H'):
        atom_positions.append(i)

for i in range(len(molecules)):
  print('BDE to H' + str(atom_positions[i] - 1) + ':' + str(627.5*bdes[i]) + ' kcal/mol')


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


BDE to H6:123.94690777580948 kcal/mol
BDE to H7:121.12403734395748 kcal/mol
BDE to H8:121.13119190087623 kcal/mol
BDE to H9:124.26168561959855 kcal/mol
